In [7]:
from astropy.table import Table
import Paths.Paths as paths
Path = paths.filepaths()
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import vstack
from astropy.coordinates import SkyCoord
from astropy import units as u

def check_overlap(image_dir1, image_dir2, cat1, cat2):
    cats = [cat1, cat2]
    catnum1 = len(cat1['peak_x_sky'])
    catnum2 = len(cat2['peak_y_sky'])
    isinside1 = np.zeros(catnum1, dtype=bool)
    isinside2 = np.zeros(catnum2, dtype=bool)
    isinsides = [isinside1, isinside2]
    image_dirs = [image_dir1, image_dir2]
    catnums = [catnum1, catnum2]
    for i, image_dir in enumerate(image_dirs):
        fitsdata = fits.open(image_dir)
        wcs = WCS(fitsdata[0].header,naxis=2)
        image = fitsdata[0].data[0][0]
        if len(image.shape)>2:
            image = fitsdata[0].data[0][0]

        cat_comp = cats[i] ; cat = cats[i-1]

        xpeak = cat['peak_x_sky']
        ypeak = cat['peak_y_sky']

        xypeak = np.vstack((xpeak,ypeak)).T

        xypix = wcs.wcs_world2pix(xypeak,0)
        
        for j in range(catnums[i-1]):
            if xypix[j,0] < 0 or xypix[j,1] < 0 or xypix[j,0]>=image.shape[0] or xypix[j,1] >= image.shape[1]:
                isinsides[i-1][j] = False
            
            else:
                isinsides[i-1][j] = np.isfinite(image[int(xypix[j,0]),int(xypix[j,1])])
                
    return isinsides

def add_skycoordinates(cat, wcs):
    """
    Add sky coordinates to the catalog based on WCS information.
    
    Parameters:
    cat : dict
        The catalog containing peak_x and peak_y pixel coordinates.
    wcs : astropy.wcs.WCS
        The WCS object for converting pixel coordinates to sky coordinates.
    
    Returns:
    None: The function modifies the input catalog in place.
    """
    xypix = np.column_stack((cat['peak_x'].data, cat['peak_y'].data))
    xysky = wcs.wcs_pix2world(xypix, 0)
    cat['peak_x_sky'] = xysky[:, 0]
    cat['peak_y_sky'] = xysky[:, 1]

def auto_matching(cat1, cat2, threshold=0.01, verbose=False):
    
    catnum1 = len(cat1['peak_x_sky'])
    matchingind = []            
    for i in range(catnum1):
        dist = np.sqrt((cat1['peak_x_sky'][i] - cat2['peak_x_sky'])**2
                      +(cat1['peak_y_sky'][i] - cat2['peak_y_sky'])**2)
        if verbose:
            print(i,cat1['_name'][i],cat2['_name'][np.argmin(dist)],np.min(dist),np.argmin(dist))
        if np.min(dist) < threshold:
            matchingind.append(np.argmin(dist))
        else:
            matchingind.append(-1)

        
    return np.array(matchingind)                       


def matching_main(b3cat, b6cat, b3fits, b6fits, threshold=2e-5,label='w51e'):
    add_skycoordinates(b3cat, WCS(fits.open(b3fits)[0].header, naxis=2))
    add_skycoordinates(b6cat, WCS(fits.open(b6fits)[0].header, naxis=2))
    isinsides = check_overlap(b3fits, b6fits,b3cat, b6cat)
    b6_to_b3_ind = auto_matching(b3cat, b6cat, threshold=threshold, verbose=False)
    b3_to_b6_ind = auto_matching(b6cat, b3cat, threshold=threshold, verbose=False)   

    b3_ind_counterpart = np.where(b6_to_b3_ind>=0)
    b3_counterpart = b3cat[b3_ind_counterpart]
    b6_counterpart = b6cat[b6_to_b3_ind[b3_ind_counterpart]]
    
    raarr = []
    decarr = []

    for i in range(len(b3_counterpart)):
        if b6_counterpart['peak_x'][i] < 0 or b6_counterpart['peak_y'][i] < 0:
            raarr.append(b3_counterpart['peak_x'][i])
            decarr.append(b3_counterpart['peak_y'][i])
        else:
            raarr.append(b6_counterpart['peak_x_sky'][i])
            decarr.append(b6_counterpart['peak_y_sky'][i])

    table_matched = Table([
                        np.array(raarr), np.array(decarr),
                        np.ones(len(b3_counterpart), dtype=bool),
                        np.ones(len(b6_counterpart), dtype=bool),
                         isinsides[0][b3_ind_counterpart]],
                    names=(  'ra', 'dec',
                            'isdetected_b3','isdetected_b6',
                            'is_overlap')  
                    )
    print(len(table_matched))                    
    b3_ind_nomatch = np.where(b6_to_b3_ind<0)
    b3only = b3cat[b3_ind_nomatch]
    b3only_num = len(b3_ind_nomatch[0])
    dummyarr = np.ones(b3only_num)*(-1)
    dummyarr_str = np.zeros(b3only_num, dtype=str)
  
    table_b3only = Table([
                        b3only['peak_x_sky'], b3only['peak_y_sky'],
                        np.ones(b3only_num, dtype=bool), np.zeros(b3only_num, dtype=bool),
                        isinsides[0][b3_ind_nomatch]],
                        names=(
                            'ra', 'dec',
                            'isdetected_b3', 'isdetected_b6',
                            'is_overlap')  
                        )

    print(len(table_b3only))                    
                        
    b6_ind_nomatch = np.where(b3_to_b6_ind<0)
    b6only = b6cat[b6_ind_nomatch]
    b6only_num = len(b6_ind_nomatch[0])
    dummyarr = np.ones(b6only_num)*(-1)
    dummyarr_str = np.zeros(b6only_num, dtype=str)
    table_b6only = Table([
                        b6only['peak_x_sky'], b6only['peak_y_sky'],
                        np.zeros(b6only_num, dtype=bool), np.ones(b6only_num, dtype=bool),
                        isinsides[1][b6_ind_nomatch]],
                        names=(
                            'ra', 'dec',
                            'isdetected_b3', 'isdetected_b6',
                            'is_overlap')  
                        )
    print(len(table_b6only))                    

    table_fin = vstack([table_matched, table_b3only, table_b6only])
   
    table_fin = vstack([table_fin[table_fin['is_overlap']], table_fin[~table_fin['is_overlap']]])






    table_fin.write(f'tables/{label}_ambiguous.fits', format='fits', overwrite=True)

    print(len(table_fin))        


for reg in ['W51-E', 'W51-IRS2']:
    ambiguous_b3 = Table.read(f'tables/{reg}_B3_ambiguous_dendro.fits')
    ambiguous_b6 = Table.read(f'tables/{reg}_B6_ambiguous_dendro.fits')
    if reg == 'W51-E':
        matching_main(ambiguous_b3, ambiguous_b6, Path.w51e_b3_tt0, Path.w51e_b6_tt0, label=reg)
    elif reg == 'W51-IRS2':
        matching_main(ambiguous_b3, ambiguous_b6, Path.w51n_b3_tt0, Path.w51n_b6_tt0, label=reg)

9
19
17
45
4
13
16
33


Set OBSGEO-B to   -23.022886 from OBSGEO-[XYZ].
Set OBSGEO-H to     5053.796 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
